#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter GA360 Segmentology Parameters
GA360 funnel analysis using Census data.
 1. Wait for <b>BigQuery->->->Census_Join</b> to be created.
 1. Join the <a hre='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/3673497b-f36f-4448-8fb9-3e05ea51842f/' target='_blank'>GA360 Segmentology Sample</a>. Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to <b>BigQuery->->->Census_Join</b>.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_write': 'service',  # Authorization used for writing data.
  'auth_read': 'service',  # Authorization for reading GA360.
  'view': 'service',  # View Id
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute GA360 Segmentology
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'description': 'Create a dataset for bigquery tables.',
      'hour': [
        4
      ],
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'function': 'Pearson Significance Test',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'ga': {
      'auth': 'user',
      'kwargs': {
        'reportRequests': [
          {
            'viewId': {'field': {'name': 'view','kind': 'string','order': 2,'default': 'service','description': 'View Id'}},
            'dateRanges': [
              {
                'startDate': '90daysAgo',
                'endDate': 'today'
              }
            ],
            'dimensions': [
              {
                'name': 'ga:userType'
              },
              {
                'name': 'ga:userDefinedValue'
              },
              {
                'name': 'ga:latitude'
              },
              {
                'name': 'ga:longitude'
              }
            ],
            'metrics': [
              {
                'expression': 'ga:users'
              },
              {
                'expression': 'ga:sessionsPerUser'
              },
              {
                'expression': 'ga:bounces'
              },
              {
                'expression': 'ga:timeOnPage'
              },
              {
                'expression': 'ga:pageviews'
              }
            ]
          }
        ],
        'useResourceQuotas': False
      },
      'out': {
        'bigquery': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'GA360_KPI'
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': 'WITH GA360_SUM AS (           SELECT              A.Dimensions.userType AS User_Type,             A.Dimensions.userDefinedValue AS User_Value,             B.zip_code AS Zip,             SUM(Metrics.users) AS Users,             SUM(Metrics.sessionsPerUser) AS Sessions,             SUM(Metrics.timeOnPage) AS Time_On_Site,             SUM(Metrics.bounces) AS Bounces,             SUM(Metrics.pageviews) AS Page_Views           FROM `{dataset}.GA360_KPI` AS A            JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` AS B           ON ST_WITHIN(ST_GEOGPOINT(A.Dimensions.longitude, A.Dimensions.latitude), B.zip_code_geom)           GROUP BY 1,2,3           )           SELECT             User_Type,             User_Value,             Zip,             Users,             SAFE_DIVIDE(Users, SUM(Users) OVER()) AS User_Percent,             SAFE_DIVIDE(Sessions, SUM(Sessions) OVER()) AS Impression_Percent,             SAFE_DIVIDE(Time_On_Site, SUM(Time_On_Site) OVER()) AS Time_On_Site_Percent,             SAFE_DIVIDE(Bounces, SUM(Bounces) OVER()) AS Bounce_Percent,             SAFE_DIVIDE(Page_Views, SUM(Page_Views) OVER()) AS Page_View_Percent           FROM GA360_SUM        ',
        'parameters': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
        },
        'legacy': False
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be written in BigQuery.'}},
        'view': 'GA360_KPI_Normalized'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'normalize': {
        'census_geography': 'zip_codes',
        'census_year': '2018',
        'census_span': '5yr'
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'correlate': {
        'join': 'Zip',
        'pass': [
          'User_Type',
          'User_Value'
        ],
        'sum': [
          'Users'
        ],
        'correlate': [
          'User_Percent',
          'Impression_Percent',
          'Time_On_Site_Percent',
          'Bounce_Percent',
          'Page_View_Percent'
        ],
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'table': 'GA360_KPI_Normalized',
        'significance': 80
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
